<a href="https://colab.research.google.com/github/ameypatil10/ASR-Project/blob/master/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/CS728_assignment2')

In [3]:
import re
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')

from collections import Counter

from nltk import word_tokenize

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from keras.utils.np_utils import to_categorical

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
embed_size = 50
max_words = 50000
SEED = 10

In [5]:
data = pd.read_csv("train.csv")

In [6]:
data['compounds']=data['flea']+' '+data['market']
data=data[['compounds','LEXICALIZED']]

In [7]:
def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)
    return x

In [8]:
data["compounds"] = data["compounds"].apply(lambda x: clean_text(x))

In [9]:
max_len=max(len(i.split(' ')) for i in data["compounds"])

In [10]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(data['compounds'], data['LEXICALIZED'],
                                                    stratify=data['LEXICALIZED'], 
                                                    test_size=0.25)

In [11]:
tokenizer = Tokenizer(max_words)
tokenizer.fit_on_texts(list(train_x))
train_x = tokenizer.texts_to_sequences(train_x)
val_x = tokenizer.texts_to_sequences(val_x)

train_x = np.asarray(pad_sequences(train_x, maxlen=max_len))
val_x = np.asarray(pad_sequences(val_x, maxlen=max_len))

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y = to_categorical(le.fit_transform(train_y.values))
val_y = to_categorical(le.transform(val_y.values))

In [13]:
no_of_classes=len(le.classes_)

In [16]:
def load_glove(word_index):
    EMBEDDING_FILE = 'glove.6B.50d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = max_words
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_words: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [17]:
embedding_matrix = load_glove(tokenizer.word_index)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
e = Embedding(max_words, embed_size, weights=[embedding_matrix], input_length=3, trainable=False)
model = Sequential()
model.add(e)
model.add(Flatten())
model.add(ReLu)
model.add(Dense(no_of_classes,activation='sigmoid'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 50)             2500000   
_________________________________________________________________
flatten (Flatten)            (None, 150)               0         
_________________________________________________________________
dense (Dense)                (None, 37)                5587      
Total params: 2,505,587
Trainable params: 5,587
Non-trainable params: 2,500,000
_________________________________________________________________


In [20]:
model.fit(train_x, train_y ,validation_data = (val_x, val_y), epochs=100)

Epoch 1/100
382/382 [==============================] - 1s 2ms/step - loss: 3.1464 - accuracy: 0.1597 - val_loss: 2.4318 - val_accuracy: 0.3098
Epoch 2/100
382/382 [==============================] - 1s 2ms/step - loss: 2.2270 - accuracy: 0.3704 - val_loss: 2.1752 - val_accuracy: 0.3832
Epoch 3/100
382/382 [==============================] - 1s 2ms/step - loss: 1.9732 - accuracy: 0.4460 - val_loss: 2.0592 - val_accuracy: 0.4235
Epoch 4/100
382/382 [==============================] - 1s 2ms/step - loss: 1.8136 - accuracy: 0.4842 - val_loss: 2.0013 - val_accuracy: 0.4390
Epoch 5/100
382/382 [==============================] - 1s 2ms/step - loss: 1.7182 - accuracy: 0.5156 - val_loss: 1.9618 - val_accuracy: 0.4537
Epoch 6/100
382/382 [==============================] - 1s 2ms/step - loss: 1.6780 - accuracy: 0.5200 - val_loss: 1.9442 - val_accuracy: 0.4611
Epoch 7/100
382/382 [==============================] - 1s 2ms/step - loss: 1.6113 - accuracy: 0.5415 - val_loss: 1.9303 - val_accuracy: 0.4665